Uncomment the following line in order to install pytorch-fit package.

In [1]:
# !pip install --use-feature=in-tree-build ./

In [2]:
%load_ext autoreload
%autoreload 1
%aimport pytorch_fit

In [3]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader

import pytorch_fit
from pytorch_fit import fit, evaluate
from pytorch_fit.metrics import Accuracy, ConfusionMatrix
from pytorch_fit.stopping import EarlyStopping
from pytorch_fit.visuals import plot_history, display_last_history,\
    display_network, format_vertical_headers

In [4]:
%matplotlib inline
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
loss = nn.BCELoss(reduction="sum")

val_loss_stopping = EarlyStopping(
    "Loss", "val_loss", n_epochs=10, stats_fun=np.max, min_change=-0.001
)
val_acc_stopping = EarlyStopping(
    "Accuracy", "val_acc", n_epochs=10, stats_fun=np.mean, min_change=0.01
)
val_stopping = val_loss_stopping.and_(val_acc_stopping)

train_loss_stopping = EarlyStopping(
    "Loss", "train_loss", n_epochs=10, stats_fun=np.max, min_change=-0.001,
    relative_change_type=True
)
train_acc_stopping = EarlyStopping(
    "Accuracy", "train_acc", n_epochs=10, stats_fun=np.max, min_change=0.01
)
train_stopping = train_loss_stopping.and_(train_acc_stopping)

stopping = train_stopping.or_(val_stopping)